In [54]:
from stravalib.client import Client
import pandas as pd
from geopy.distance import vincenty
from retrying import retry
import time
import numpy as np
from tqdm import tqdm

client = Client(access_token='76824abf6abf903eb3d8b0bde83625135c0be0ec')
athlete = client.get_athlete()

In [55]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_leaderboard(client,segment):
    total = client.get_segment_leaderboard(segment)
    friends = client.get_segment_leaderboard(segment,following=True)
    c = (total,friends)
    return c

@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_segment(client,segment):
    return client.get_segment(segment)

In [56]:
df = pd.read_csv('segoutput.csv',index_col=False)

home = (45.487803,-73.576913)
df['coordinates'] = zip(df['latitude'], df['longitude'])

biglist = df.T.to_dict().values()
for row in biglist:
    row['distance_from_home'] = vincenty(row['coordinates'], home).km

new_df = pd.DataFrame(biglist)
new_df = new_df[new_df['distance_from_home'] <= 2].sort_values(['distance_from_home'], ascending=[True])

smalllist = new_df.T.to_dict().values()

for row in tqdm(smalllist):
    
    time.sleep(2)
        
    row['segment_distance'] = float(retry_get_segment(client,row["segment_id"]).distance)
    
    both_leaderboards = retry_get_leaderboard(client,row["segment_id"]) 
    row['total_top_time'] = both_leaderboards[0][0].elapsed_time
    
    times = both_leaderboards[0][0].elapsed_time
    for x in both_leaderboards[0][1:]:
        times += x.elapsed_time
    times = times / len(both_leaderboards[0])
    row['average_total_top_time'] = times
    
    
    if not both_leaderboards[1]:
        row['friend_top_time'] = np.nan        
    else:
        row['friend_top_time'] = both_leaderboards[1][0].elapsed_time
        
final_df = pd.DataFrame(smalllist)
final_df['friend_versus_top1'] = (final_df['friend_top_time'] - final_df['total_top_time'])/final_df['total_top_time']*100
final_df['friend_versus_top10'] = (final_df['friend_top_time'] - final_df['average_total_top_time'])/final_df['average_total_top_time']*100


In [65]:
final_df['distance_from_home_Ranked'] = final_df['distance_from_home'].rank(ascending=1)
final_df['segment_distance_Ranked'] = final_df['segment_distance'].rank(ascending=1)
final_df['friend_versus_top10_Ranked'] = final_df['friend_versus_top10'].rank(ascending=0)
final_df['target_score'] = 0.2*final_df['distance_from_home_Ranked'] + 0.1*final_df['segment_distance_Ranked'] + 0.7*final_df['friend_versus_top10_Ranked']
print final_df.sort_values(['target_score'], ascending=[True])

    average_total_top_time   color                      coordinates  \
61                00:00:17  FFEE33          (45.490744, -73.594655)   
20                00:01:12  FFEE33          (45.483232, -73.579054)   
0          00:00:22.500000  FFEE33          (45.502192, -73.577163)   
56         00:00:18.600000  C7258E          (45.477057, -73.571581)   
86         00:00:23.933333  814A19          (45.495043, -73.569366)   
44         00:00:14.100000  FFEE33          (45.490763, -73.596486)   
35         00:00:39.800000  814A19  (45.4858228192, -73.5806700122)   
51         00:00:20.200000  FFEE33  (45.5003406946, -73.5856961366)   
48                00:00:20  814A19           (45.49419, -73.567444)   
81         00:01:06.400000  FFEE33  (45.4914261214, -73.5879158322)   
74         00:00:36.100000  FFEE33          (45.492301, -73.598682)   
69                00:00:36  B7E628  (45.4903114121, -73.5572500527)   
36                00:00:52  814A19          (45.485434, -73.580475)   
2     

In [43]:
final_df

,average_total_top_time,color,coordinates,distance_from_home,friend_top_time,id,latitude,longitude,name,segment_distance,segment_id,segment_name,total_top_time,type,url
0,00:00:22.500000,FFEE33,"(45.502192, -73.577163)",1.599334,00:00:58,1537,45.502192,-73.577163,Peel Push - Sherbrooke to Docteur Penfield : ...,204.300,7849363,Peel Push - Sherbrooke to Docteur Penfield,00:00:19,Sasha Wloski,http://www.strava.com/segments/7849363/compare/
1,00:00:23.500000,A03D3D,"(45.5021951906, -73.5629598051)",1.936012,00:01:14,1538,45.502195,-73.562960,Beaver Hall: [Laurie McQueen],237.550,1176830,Beaver Hall,00:00:20,Laurie McQueen,http://www.strava.com/segments/1176830/compare/
2,00:00:51.400000,FFEE33,"(45.4916093499, -73.5882798583)",0.984138,00:01:25,1539,45.491609,-73.588280,Atwater/Saint Sulpice Short Safe: [Sasha Wloski],411.851,1840013,Atwater/Saint Sulpice Short Safe,00:00:40,Sasha Wloski,http://www.strava.com/segments/1840013/compare/
3,00:00:58.400000,9CD0EF,"(45.49695, -73.586141)",1.246526,00:01:15,1540,45.496950,-73.586141,DrPenfield: [Eoin Craigie],836.300,7381906,DrPenfield,00:00:53,Eoin Craigie,http://www.strava.com/segments/7381906/compare/
